# DRAG Quadrature Scaling Factor Calibration

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see [our tutorials](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/index.html) if you need to create your setup and qubits for the first time. 

You can run this notebook on real hardware in the lab. However, if you don't have the hardware at your disposal, you can also run the notebook "as is" using an emulated session (see below). 

If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this experiment, we determine the quadrature scaling factor, $\beta$, of a Derivative Removal by Adiabatic Gate (DRAG) pulse, which is optimal for cancelling dynamics phase errors that occur during the application of a qubit drive pulse. The DRAG drive pulse has the following form:

$$v(t)=i(t) + q(t),$$

where the quadrature component $q(t)$ is given by the derivative of the in-phase component $i(t)$, scaled by a factor $\beta$:

$$q(t) = \beta \frac{d(i(t))}{d(t)}$$

In order to determine the optimal $\beta$ for compensating phase errors, we apply a pulse sequence that is sensitive to phase errors and sweep the value of $\beta$ for all the drive pulses in the sequence. As shown in the image below, we use pairs of drive pulses followed by a measurement.

![](../../../images/drag_sequence.svg "Quantum circuit diagram from the DRAG pulse calibration. RO refers to the measurement.")

In the absence of phase errors, all three of these sequences of two pulses should move the state vector of a qubit prepared in the ground state onto the equator, along the $\pm \hat{y}$ axis of the Bloch sphere. However, phase errors introduce unwanted rotations around the $\hat{z}$ axis during the application of the pulse, resulting in the qubit state vector ending up away from the equatorial plane for the pulse pairs `x90-y180` and `x90-ym180` as shown below:

![](../../../images/drag_bloch.svg "Cartoon of the qubit state vectors on the Bloch sphere after the first x90 (black) gate and after the subsequent second gate (colors) in the presence of phase errors.")

The goal of the calibration experiment is to find the quadrature scaling factor $\beta$ that results in all three pairs of pulses correctly moving the qubit state vector onto the equator as shown in the following plot:

![](../../../images/drag_plot.svg "Excited-state population after applying the pulse sequences as a function of the scaling factor with a second-order DRAG. The colored markers and lines are correspond to the respective pulse sequences. The vertical line indicates the optimal value for cancelling phase errors.")


In this guide, we refer to the $\beta$ parameter as a "q-scaling". If multiple qubits are passed to the experiment, the above pulses are applied in parallel on all the qubits.

### Imports

You'll start by importing the DRAG quadrature-scaling calibration experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo quantum platform containing a qpu and device setup to run in emulation mode. See the [Getting Started tutorial](../../../../tutorials/sources/getting_started.ipynb) for more details about the quantum platform and how to create your experimental setup.

In [ ]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import drag_q_scaling
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup as shown in the [Getting Started tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/getting_started.html).

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Create a `FolderStore` for Saving Data

The experiment `Workflows` can automatically save the inputs and outputs of all their tasks to the folder path we specify when instantiating the `FolderStore`. Here, we choose the current working directory.

In [ ]:
# import FolderStore from the `workflow` namespace of LabOne Q, which was imported
# from `laboneq.simple`
from pathlib import Path

folder_store = workflow.logbook.FolderStore(Path.cwd())

We disable saving in this guide. To enable it, simply run `folder_store.activate()`.

In [ ]:
folder_store.deactivate()

### Running the Experiment Workflow

You'll now make the experiment workflow and run. For more details on what experiment workflows are and what tasks they execute, see the [Experiment Workflows tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/experiment_workflows.html).

Let's first create the options class for the DRAG calibration experiment and inspect it using the tree view of the option fields per task:

In [ ]:
options = drag_q_scaling.experiment_workflow.options()
options

Or, alternatively, using the `show_fields` function from the `workflow` namespace of LabOne Q, which was imported from `laboneq.simple`:

In [ ]:
workflow.show_fields(options)

Notice that, unless we change it:

- the experiment is run in `AcquisitionType.INTEGRATION` and `AveragingMode.CYCLIC`, using 1024 averages (`count`)
- the experiment is run on the qubit $|g\rangle \leftrightarrow |e\rangle$ (`transition`)
- calibration traces are added (`use_cal_traces=True`) for the states $|g\rangle$ and $|e\rangle$ (`cal_states`)
- the analysis workflow will run automatically (`do_analysis=True`)
- the figures produced by the analysis are automatically closed (`close_figures=False`)
- the qubit parameters will not be updated (`update=False`)

Here, let's disable closing the figures:

In [ ]:
options.close_figures(False)

Now we run the experiment workflow on the first two qubits in parallel.

Note that the fit fails in emulation mode and the quadrature scaling factor cannot be extracted.

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

exp_workflow = drag_q_scaling.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    q_scalings=[np.linspace(-0.02, 0.02, 11), np.linspace(-0.03, 0.03, 11)],
    options=options
)

workflow_results = exp_workflow.run()

#### Inspect the Tasks That Were Run

In [ ]:
for t in workflow_results.tasks:
    print(t)

#### Inspect the Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_experiment = workflow_results.tasks["compile_experiment"].output
plot_simulation(compiled_experiment, length=50e-6)

#### Inspecting the Source Code of the Pulse-Sequence Creation Task

You can inspect the source code of the `create_experiment` task defined in `drag_q_scaling` to see how the experiment pulse sequence is created using quantum operations. To learn more about the latter, see the [Quantum Operations tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/quantum_operations.html).

In [ ]:
drag_q_scaling.create_experiment.src

To learn more about how to work with experiment `Workflows`, check out the [Experiment Workflows tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/experiment_workflows.html).

Here, let's briefly inspect the analysis-workflow results.

#### Analysis Results

Let's check what tasks were run as part of the analysis workflow:

In [ ]:
analysis_workflow_results = workflow_results.tasks["analysis_workflow"]
for t in analysis_workflow_results.tasks:
    print(t)

We can access the qubit parameters extracted by the analysis from the output of the analysis-workflow. However, because the fit fails, the quadrature scaling factor cannot be extracted in emulation mode, and the new qubit parameters are empty.

In [ ]:
from pprint import pprint

pprint(analysis_workflow_results.output)  # noqa: T203

Check out the [Experiment Workflows tutorial](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/experiment_workflows.html) to see how you can manually update the qubit parameters to these new values, or reset them to the old ones. 

Great! You've now run the DRAG quadrature-scaling calibration experiment. Check out other experiments in this manual to keep characterizing your qubits.